### Imports

In [38]:
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
import json

### Read stuff ###

In [42]:
data = pd.read_csv("chix.csv", sep = ";")

In [3]:
c = list(pd.unique(data["Contra Broker"].dropna()))
b = list(pd.unique(data["Broker"].dropna()))

for num in c:
    if num not in b:
        b.append(num)

brokers = []
for num in b:
    if num != np.nan and num != 1.0:
        brokers.append(int(num))

In [4]:
count_contrabro = {}

for bro_num in brokers:
    contrabro = data[data["Contra Broker"] == bro_num]
    count = len(contrabro)
    count_contrabro[bro_num] = count

In [30]:
notionals = {84: 0,
 65: 24810000,
 2: 24830000,
 124: 24880000,
 85: 49540000,
 33: 74400000,
 88: 125350000,
 70: 126100000,
 19: 148800000,
 79: 198560000,
 7: 991200000,
 14: 1033200000,
 72: 1170300000,
 80: 1437360000,
 53: 1908720000,
 9: 3519370000,
 39: 70971540000}


trans_scores = {
 2: 3.1827256438833875,
 7: -1.3573270227856293,
 9: -4.00035673529397,
 14: -59.91700133868267,
 19: 2.8735638560042704,
 33: 6.959353882165052,
 39: 15.146893484193013,
 53: 0.8881142037841788,
 65: -2.0525839668127177,
 70: -3.8065268065272213,
 72: 40.916586768929236,
 79: 1.4556451093714489,
 80: -2.971875998372899,
 84: 0,
 85: 0.8197709542841985,
 88: 0.5141737032573598,
 124: -0.17830020022881654}

In [6]:
exec = data[data["Message Type"] == "E"]
trade = {}

for bro_num in brokers:
    bro_exec = exec[exec["Broker"] == bro_num]
    count = len(bro_exec)
    trade[bro_num] = count

for key in trade:
    new_value = trade[key] + count_contrabro[key]
    trade[key] = new_value

trade

{80: 806,
 79: 899,
 7: 29,
 2: 168,
 39: 65,
 9: 135,
 33: 11,
 85: 35,
 53: 73,
 72: 1,
 84: 0,
 19: 7,
 70: 1,
 14: 4,
 88: 1,
 124: 2,
 65: 5}

In [7]:
volume = {}

for bro_num in brokers:
    bro_exec1 = exec[exec["Broker"] == bro_num]
    bro_exec2 = exec[exec["Contra Broker"] == bro_num]
    total = bro_exec1["Executed Shares"].sum() + bro_exec2["Executed Shares"].sum()
    volume[bro_num] = total

volume

{80: 1945900.0,
 79: 2140000.0,
 7: 32700.0,
 2: 716000.0,
 39: 298200.0,
 9: 491600.0,
 33: 11700.0,
 85: 13400.0,
 53: 47800.0,
 72: 4700.0,
 84: 0.0,
 19: 9700.0,
 70: 500.0,
 14: 18400.0,
 88: 500.0,
 124: 200.0,
 65: 1000.0}

In [8]:
ratio = {}

for key in notionals:
    try:
        r = notionals[key]/trade[key]
        ratio[key] = r
    
    except ZeroDivisionError:
        ratio[key] = 0

ratio

{84: 0,
 65: 4962000.0,
 2: 147797.61904761905,
 124: 12440000.0,
 85: 1415428.5714285714,
 33: 6763636.363636363,
 88: 125350000.0,
 70: 126100000.0,
 19: 21257142.85714286,
 79: 220867.63070077865,
 7: 34179310.344827585,
 14: 258300000.0,
 72: 1170300000.0,
 80: 1783325.0620347394,
 53: 26146849.315068495,
 9: 26069407.407407407,
 39: 1091869846.1538463}

In [29]:
len(trans_scores)

18

In [31]:
criterias = [count_contrabro, notionals, trans_scores, trade, volume, ratio]

In [32]:
size = len(brokers)
initial_scores = [0.0] * size
scores = dict(zip(brokers, initial_scores))

In [35]:
for crit in criterias:
    crit_val = list(crit.values())
    mean = np.mean(crit_val)
    std = np.std(crit_val)

    for bk in crit:
        z = (crit[bk] - mean) * (size ** 0.5)/std
        scores[bk] += z

scores

{80: 65.74340775377539,
 79: 45.5894477493906,
 7: -15.675508440671953,
 2: 4.9470953497916845,
 39: 55.54138760343744,
 9: -5.626134422132532,
 33: -13.525128981137346,
 85: -15.876963800572856,
 53: -12.415593464151861,
 72: 28.541046424853292,
 84: -17.741359593403303,
 19: -15.203565384126534,
 70: -16.40920940140422,
 14: -37.915797453351345,
 88: -14.46550106402884,
 124: -17.364027195626885,
 65: -18.143595680640672}